In [2]:
import numpy as np
import pandas as pd
from collections import Counter
from scipy.stats import kurtosis,skew
from time import time
from sklearn import cross_validation as model_selection
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm

/Users/adithya8.0.0/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


u'/Users/adithya8.0.0/Desktop/HAR'

In [3]:
data = pd.read_pickle("accelero_data.pkl")
lab = pd.read_pickle("activity_label.pkl")

In [15]:
data = np.array(data)
lab = np.array(lab)
print data.shape
arr = np.hstack((data,lab))
print arr.shape

(815614, 3)
(815614, 4)


In [12]:
for i in xrange(1,13):
    print i," : ",np.count_nonzero(lab == i)

1  :  122091
2  :  116707
3  :  107961
4  :  126677
5  :  138105
6  :  136865
7  :  10316
8  :  8029
9  :  12428
10  :  11150
11  :  14418
12  :  10867


In [31]:
attributes = []
start=time()
act = []
#act = np.array(act)
l = 0
i=0
t = time()
while i < len(arr):
    print i
    if i+51>len(arr):
        break
    if(arr[i,-1] != arr[i+51,-1]):
        i+=1
        continue
    else:
        attr_mean = np.mean(arr[i:i+52,:-1],axis = 0)
        attr_var = np.var(arr[i:i+52,:-1],axis=0)
        attr_min = np.amin(arr[i:i+52,:-1],axis = 0)
        attr_max = np.amax(arr[i:i+52,:-1],axis = 0)  
        attr_skew = skew(arr[i:i+52,:-1],axis=0)
        attr_kurtosis = kurtosis(arr[i:i+52,:-1],axis=0)
        
        attr_coeff = []
        
        for j in [0]:  #[0,4,8]
            attr_coeff.append(np.corrcoef(arr[i:i+52,j+0],arr[i:i+52,j+1])[1,0])
            attr_coeff.append(np.corrcoef(arr[i:i+52,j+1],arr[i:i+52,j+2])[1,0])
            attr_coeff.append(np.corrcoef(arr[i:i+52,j+2],arr[i:i+52,j+0])[1,0])
        attr_coeff = np.array(attr_coeff)
        
        attr = attr_mean
        attr = np.concatenate((attr,attr_var))
        attr = np.concatenate((attr,attr_coeff))        
        attr = np.concatenate((attr,attr_min))
        attr = np.concatenate((attr,attr_max))
        attr = np.concatenate((attr,attr_skew))
        attr = np.concatenate((attr,attr_kurtosis))

        
        act.append(arr[i,-1])
    
        attributes.append(attr)
        del attr
        i+=26
print time() - start        
start = time()
attributes = np.array(attributes)

0
26
52
78
104
130
156
182
208
234
260
286
312
338
364
390
416
442
468
494
520
546
572
598
624
650
676
702
728
754
780
806
832
858
884
910
936
937
938
939
940
941
942
943
944
945
946
947
948
949
950
951
952
953
954
955
956
957
958
959
960
961
962
963
964
965
966
967
968
969
970
971
972
973
974
975
976
977
978
979
980
981
982
983
1009
1035
1061
1087
1113
1114
1115
1116
1117
1118
1119
1120
1121
1122
1123
1124
1125
1126
1127
1128
1129
1130
1131
1132
1133
1134
1135
1136
1137
1138
1139
1140
1141
1142
1143
1169
1195
1221
1247
1273
1299
1325
1351
1377
1403
1429
1455
1481
1507
1533
1559
1585
1611
1637
1663
1689
1715
1741
1767
1793
1819
1845
1871
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1971
1997
2023
2049
2075
2076
2077
2078
2079
2080
2081
2082
2083
2084
2085
2086
2087
2088
2089
2090
2091
2092

In [32]:
print attributes.shape, np.shape(act)

(29687, 21) (29687,)


In [37]:
def efficiency(feats):
    num_est=5
    columns_list=["Mean x","Mean y","Mean z",
              "Dev x","Dev y","Dev z",
              "Corr x-y","Corr x-z","Corr y-z",
              "Min x","Min y","Min z",
             "Max x","Max y","Max z",
             "Skew x","Skew y","Skew z",
             "Kurtosis x", "Kurtosis y","Kurtosis z"]

    X_train, X_test, y_t, y_test = model_selection.train_test_split(feats, act, test_size=0.25, random_state=0, stratify = act)

    print ("For Decision Trees:")
    d_tree=DecisionTreeClassifier()
    d_tree.fit(X_train,y_t)
    t1=time()
    print (d_tree.score(X_test,y_test))
    print ("Time Taken : " + str(time()-t1))
    print()

    print ("For SVM:")
    clf=svm.SVC()
    clf.fit(X_train,y_t)
    t1=time()
    print (clf.score(X_test,y_test))
    print ("Time Taken : " + str(time()-t1))
    print ()
    
    print ("For Naive bayes:")
    gnb=GaussianNB()
    gnb.fit(X_train,y_t)
    t1=time()
    print (gnb.score(X_test,y_test))
    print ("Time Taken : " + str(time()-t1))
    print()



In [35]:
feats = attributes
print ("For Random Forest:")

X_train, X_test, y_t, y_test = model_selection.train_test_split(feats, act, test_size=0.25, random_state=42,stratify=act)
for cri in ["entropy","gini"]:
    print ("Criterion : ",cri)
    rf=RandomForestClassifier(n_estimators=100,max_features=6,criterion=cri)
    rf.fit(X_train,y_t)
    t1=time()
    print (rf.score(X_test,y_test))
    print ("Time Taken : " + str(time()-t1))
    print (classification_report(y_test,rf.predict(X_test),digits=4))

print ("For Extra Trees:")

X_train, X_test, y_t, y_test = model_selection.train_test_split(feats, act, test_size=0.25, random_state=42,stratify=act)
for cri in ["entropy","gini"]:
    print ("Criterion : ",cri)	
    rf=ExtraTreesClassifier(n_estimators=100,max_features=6,criterion=cri)
    rf.fit(X_train,y_t)
    t1=time()
    print (rf.score(X_test,y_test))
    print ("Time Taken : " + str(time()-t1))
    print (classification_report(y_test,rf.predict(X_test),digits=4))


For Random Forest:
('Criterion : ', 'entropy')
0.924548639181
Time Taken : 0.284849882126
             precision    recall  f1-score   support

        1.0     0.9473    0.9514    0.9493      1152
        2.0     0.9189    0.9564    0.9373      1055
        3.0     0.9574    0.9485    0.9529       971
        4.0     0.9634    0.9650    0.9642      1173
        5.0     0.9590    0.9665    0.9628      1284
        6.0     0.9829    0.9969    0.9899      1272
        7.0     0.5000    0.3896    0.4380        77
        8.0     0.5455    0.4364    0.4848        55
        9.0     0.5051    0.5155    0.5102        97
       10.0     0.4810    0.4471    0.4634        85
       11.0     0.3246    0.3109    0.3176       119
       12.0     0.3200    0.1951    0.2424        82

avg / total     0.9196    0.9245    0.9217      7422

('Criterion : ', 'gini')
0.919024521692
Time Taken : 0.255610942841
             precision    recall  f1-score   support

        1.0     0.9338    0.9427    0.9382 

In [38]:
efficiency(attributes)

For Decision Trees:
0.865534896254
Time Taken : 0.00551295280457
()
For SVM:
0.844785772029
Time Taken : 3.7008471489
()
For Naive bayes:
0.775801670709
Time Taken : 0.029009103775
()
